Copyright © 2020, SAS Institute Inc., Cary, NC, USA.  All Rights Reserved.
SPDX-License-Identifier: Apache-2.0

# Fleet Maintenance H2O: Build and Import Trained Models into SAS Model Manager

This notebook provides an example of how to build and train a Python model and then import the model into SAS Model Manager using the fleet maintenance data set. Lines of code that must be modified by the user, such as directory paths are noted with the comment "_Changes required by user._".

_**Note:** If you download only this notebook and not the rest of the repository, you must also download the fleet_maintenance.csv from the data folder in the examples directory. These files are used when executing this notebook example._

Here are the steps shown in this notebook:

1. Import and review data and preprocess for model training.
2. Build, train, and access a decision tree, random forest, and gradient boosting model.
3. Serialize the models into separate pickle files.
4. Write the metadata JSON files needed for importing into SAS Model Manager as well as optional files for fit statistics and ROC/Lift charts.
4. Write a score code Python file for model scoring.
5. Zip the pickle, JSON, and score code files into an archive file.
6. Import the ZIP archive file to SAS Model Manager via the Session object and relevant function call.

### Python Package Imports

In [1]:
# Dataframes for data manipulations
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'
# Mathematical calculations and array handling
import numpy as np
# File handling and reading
import json
import gzip
import zipfile
import io

# H2O models
import h2o
from h2o.estimators.glm import H2OGeneralizedLinearEstimator

# Embedded plotting
import matplotlib.pyplot as plt 
plt.rc("font", size=14)

# Pathing support
from pathlib import Path

In [2]:
# sasctl interface for importing models
import sasctl.pzmm as pzmm
from sasctl import Session
from sasctl.services import model_repository as modelRepo

In [3]:
# Use 2 CPU cores & 4 GB of RAM locally
h2o.init(nthreads=2, max_mem_size=4)

Checking whether there is an H2O instance running at http://localhost:54321 . connected.


H2O_cluster_uptime:,2 hours 45 mins
H2O_cluster_timezone:,America/New_York
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.32.0.2
H2O_cluster_version_age:,2 months and 25 days
H2O_cluster_name:,H2O_from_python_sclind_u3jhan
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.987 Gb
H2O_cluster_total_cores:,16
H2O_cluster_allowed_cores:,2
H2O_cluster_status:,"locked, healthy"


### Import Data Set

In [4]:
fleetData = h2o.import_file('data/fleet_maintenance.csv')
fleetData.head(rows=3)

Parse progress: |█████████████████████████████████████████████████████████| 100%


Maintenance_flag,Speed_sensor,Vibration,Engine_Load,Coolant_Temp,Intake_Pressure,Engine_RPM,Speed_OBD,Intake_Air,Flow_Rate,Throttle_Pos,Voltage,Ambient,Accel,Engine_Oil_Temp,Speed_GPS,GPS_Longitude,GPS_Latitude,GPS_Bearing,GPS_Altitude,Turbo_Boost,Trip_Distance,Litres_Per_km,Accel_Ssor_Total,CO2,Trip_Time,CO_emission,HC_emission,PM_emission,NOx_emission,CO2_emission,Fuel_level,Oil_life,Vibration_alert,VibrationAlert_Total,Vibration_Recent,Turbo_alert,Emission_alert,Fog_control,Engine_control
0,35,249.189,21.5686,88,116,1115.5,35,10,18.33,80,14.1,7,27.8431,85,36.216,9.1417,48.9328,75.2,164,2.75572,310.262,2.3515,0.045858,62.1972,11539,0,0,0,0,0,0,0,1,123,12,1,1,1,1
0,142,243.237,20.3922,88,135,1782.5,142,16,35.41,80,14.1,8,34.5098,85,148.968,9.88538,48.4948,274.4,436,5.51143,161.025,1.24465,0.043655,32.9209,4242,0,0,0,0,0,0,0,1,123,12,1,1,1,1
0,128,244.015,43.5294,81,109,1588,128,9,27.08,80,14.2,8,14.902,79,132.408,9.64806,48.4796,257.1,508,1.74045,158.238,2.1241,0.073833,56.1824,4146,0,0,0,0,0,0,0,1,123,12,1,1,1,1


### Preprocess Data

In [5]:
fleetData['Maintenance_flag'] = fleetData['Maintenance_flag'].asfactor()

train, validation, test = fleetData.split_frame(ratios=[.6, .2], seed=42)

y = 'Maintenance_flag'
x = list(fleetData.columns)
x.remove(y)

### Create, Train, and Assess Model

In [6]:
# Generate the generalized linear estimator model called glmFit and train it on the train partition
glmFit = H2OGeneralizedLinearEstimator(family='binomial', model_id='glmFit', 
                                       lambda_search=True)
glmFit.train(x=x, y=y, training_frame=train, validation_frame=validation)

C:\Users\sclind\AppData\Local\Continuum\anaconda3\envs\dev-sasctl\lib\site-packages\h2o\estimators\estimator_base.py:200: RuntimeWarning: Dropping bad and constant columns: [PM_emission, CO2_emission, CO_emission, Vibration_alert, Fuel_level, Oil_life, Emission_alert, Fog_control, HC_emission, VibrationAlert_Total, Engine_control, NOx_emission, Turbo_alert, Vibration_Recent]
  warnings.warn(mesg["message"], RuntimeWarning)


glm Model Build progress: |███████████████████████████████████████████████| 100%


In [7]:
# Check the model performance and print its accuracy
glmPerf = glmFit.model_performance(test)
print(glmPerf.accuracy())

[[0.2722683116304135, 0.8069427527405603]]


In [8]:
mojoPath = str(Path.cwd() / 'data/FleetMaintenanceModels/GLMH2OSimple/')
glmFit.save_mojo(mojoPath, force=True)

'C:\\Users\\sclind\\Documents\\Python Scripts\\GitLab\\python-sasctl\\examples\\data\\FleetMaintenanceModels\\GLMH2OSimple\\glmFit.zip'

In [9]:
# gzip the mojo file to transfer more easily in SAS Model Manager
with open(mojoPath + '\\glmFit.zip', 'rb') as fileIn, gzip.open(mojoPath + '\\glmFit.mojo', 'wb') as fileOut:
    fileOut.writelines(fileIn)

In [10]:
# Import the mojo file into H2O and run a prediction with the test dataset
importModel = h2o.import_mojo(mojoPath + '\\glmFit.zip')
predictions = importModel.predict(test)
print(predictions)

generic Model Build progress: |███████████████████████████████████████████| 100%
Model Details
H2OGenericEstimator :  Import MOJO Model
Model Key:  Generic_model_python_1613148709495_29


GLM Model: summary


,,family,link,regularization,lambda_search,number_of_predictors_total,number_of_active_predictors,number_of_iterations,training_frame
0,,binomial,logit,"Elastic Net (alpha = 0.5, lambda = 0.003702 )","nlambda = 100, lambda.max = 0.2673, lambda.min = 0.003702, lambda....",25,24,94,py_4_sid_92e9




ModelMetricsBinomialGLMGeneric: generic
** Reported on train data. **

MSE: 0.11970005630976009
RMSE: 0.3459769592180382
LogLoss: 0.3601253555062796
Null degrees of freedom: 5010
Residual degrees of freedom: 4986
Null deviance: 5327.303889273391
Residual deviance: 3609.1763128839343
AIC: 3659.1763128839343
AUC: 0.8740980211847207
AUCPR: 0.5702749410637339
Gini: 0.7481960423694414

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.2572011221149892: 


,,0,1,Error,Rate
0,0,3110.0,780.0,0.2005,(780.0/3890.0)
1,1,189.0,932.0,0.1686,(189.0/1121.0)
2,Total,3299.0,1712.0,0.1934,(969.0/5011.0)



Maximum Metrics: Maximum metrics at their respective thresholds


...

,,timestamp,duration,iteration,lambda,predictors,deviance_train,deviance_test,training_rmse,training_logloss,...,training_pr_auc,training_lift,training_classification_error,validation_rmse,validation_logloss,validation_r2,validation_auc,validation_pr_auc,validation_lift,validation_classification_error
0,,2021-02-12 14:37:45,0.000 sec,1,.27E0,1,1.063122,1.061598,,,...,,,,,,,,,,
1,,2021-02-12 14:37:45,0.004 sec,3,.24E0,5,1.048855,1.047204,,,...,,,,,,,,,,
2,,2021-02-12 14:37:45,0.008 sec,5,.22E0,5,1.035905,1.034183,,,...,,,,,,,,,,
3,,2021-02-12 14:37:45,0.012 sec,7,.2E0,5,1.024723,1.022930,,,...,,,,,,,,,,
4,,2021-02-12 14:37:45,0.016 sec,9,.18E0,5,1.015070,1.013207,,,...,,,,,,,,,,
5,,2021-02-12 14:37:45,0.021 sec,11,.17E0,6,1.006100,1.004124,,,...,,,,,,,,,,
6,,2021-02-12 14:37:45,0.026 sec,13,.15E0,6,0.995571,0.993271,,,...,,,,,,,,,,
7,,2021-02-12 14:37:45,0.031 sec,15,.14E0,7,0.985972,0.983486,,,...,,,,,,,,,,
8,,2021-02-12 14:37:45,0.035 sec,17,.13E0,8,0.974664,0.971717,,,...,,,,,,,,,,
9,,2021-02-12 14:37:45,0.040 sec,19,.12E0,8,0.964305,0.960871,,,...,,,,,,,,,,



See the whole table with table.as_data_frame()

Variable Importances: 


,variable,relative_importance,scaled_importance,percentage
0,GPS_Altitude,1.259765,1.000000,0.161117
1,Trip_Distance,0.806042,0.639835,0.103088
2,GPS_Longitude,0.757860,0.601588,0.096926
3,Flow_Rate,0.626926,0.497653,0.080180
4,Ambient,0.602574,0.478322,0.077066
5,Trip_Time,0.550254,0.436791,0.070374
6,GPS_Latitude,0.508591,0.403719,0.065046
7,Engine_Oil_Temp,0.504403,0.400394,0.064510
8,Voltage,0.454727,0.360961,0.058157
9,Litres_Per_km,0.187535,0.148865,0.023985



See the whole table with table.as_data_frame()

generic prediction progress: |████████████████████████████████████████████| 100%


predict,p0,p1
0,0.962103,0.0378967
0,0.96441,0.0355896
0,0.992403,0.00759678
0,0.968775,0.0312247
1,0.704515,0.295485
0,0.880342,0.119658
0,0.871856,0.128144
0,0.9621,0.0378999
0,0.93915,0.0608499
0,0.937026,0.0629742


### Register Model in SAS Model Manager with pzmm

In [11]:
modelPrefix = 'glmFit'
zipFolder = Path.cwd() / 'data/FleetMaintenanceModels/GLMH2OSimple/'

traindf = train.as_data_frame()

In [12]:
# Write the input and output variable JSON files
J = pzmm.JSONFiles()
J.writeVarJSON(traindf[x], isInput=True, jPath=zipFolder)
outputVar = pd.DataFrame(columns=['EM_EVENTPROBABILITY',
                                  'EM_CLASSIFICATION'])
outputVar['EM_CLASSIFICATION'] = traindf[y].astype('category').cat.categories.astype('str')
outputVar['EM_EVENTPROBABILITY'] = 0.5
J.writeVarJSON(outputVar, isInput=False, jPath=zipFolder)

# Write the model properties JSON file
J.writeModelPropertiesJSON(modelName=modelPrefix,
                           modelDesc='',
                           targetVariable=y,
                           modelType='Classification',
                           modelPredictors=x,
                           targetEvent=1,
                           numTargetCategories=1,
                           eventProbVar='EM_EVENTPROBABILITY',
                           jPath=zipFolder,
                           modeler='sclind')

J.writeFileMetadataJSON(modelPrefix=modelPrefix,
                        jPath=zipFolder,
                        isH2OModel=True)

In [13]:
username = 'edmdev'#getpass.getpass()
password = 'Go4thsas'#getpass.getpass()
host = 'summer.edmt.sashq-d.openstack.sas.com'
sess = Session(host, username, password, protocol='http')

In [14]:
I = pzmm.ImportModel()
with sess:
    I.pzmm_import_model(zipFolder, modelPrefix, 'EDMMMX-4997', traindf[x], traindf[y],
                        '{}.predict({})', isH2OModel=True)

In [ ]:
S = pzmm.ScoreCode()
for (prefix, path) in zip(modelPrefix, zipFolder):
    zipIOFile = pzmm.ZipModel.zipFiles(_, path, prefix)
    with sess:
        modelReponse = modelRepo.import_model_from_zip(prefix, 'Fleet Management Report', zipIOFile)
        S.writeScoreCode(x, y, prefix,
                     '{}.predict({})', prefix + '.pickle',
                     pyPath=path, model=modelResponse.id)

In [ ]:
import sasctl

In [ ]:
pyPath = zipFolder / f'{modelPrefix}Score.py'
modelID = '389323fb-8812-4a93-b07c-316102e9c2ab'
with open(pyPath, 'r') as pFile:
    files = [{'name': f'{modelPrefix}Score.py', 'file': pFile, 'role': 'score'}]
    with sess:
        sasctl.tasks.upload_and_copy_score_resources(modelID, files)